In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import model_selection, preprocessing
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split

from sklearn import svm 
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
dt = pd.read_csv('neuland@jade4j.csv')

dt.head()


,travisJobId,testName,index,duration,count,failures,errors,skipped
0,87853794,de.neuland.jade4j.exceptions.JadeExceptionTest,0,0.170,2,1,0,0
1,87853794,de.neuland.jade4j.template.JadeRunFullTemplate...,1,0.050,2,0,0,0
2,87853794,de.neuland.jade4j.template.JadeConfigurationTest,2,0.013,5,0,0,0
3,87853794,de.neuland.jade4j.model.JadeModelTest,3,0.000,2,0,0,0
4,87853794,de.neuland.jade4j.parser.ComplexIndentOutdentP...,4,0.015,1,0,0,0


In [ ]:
dt.tail()


,travisJobId,testName,index,duration,count,failures,errors,skipped
35882,14243467,de.neuland.jade4j.lexer.token.TagTest,31,0.001,1,0,0,0
35883,14243467,de.neuland.jade4j.lexer.token.TextTest,32,0.003,1,0,0,0
35884,14243467,de.neuland.jade4j.lexer.WorkTest,33,0.001,1,0,0,0
35885,14243467,de.neuland.jade4j.lexer.JadeLexerTest,34,0.002,3,0,0,0
35886,14243467,de.neuland.jade4j.lexer.LexerTest,35,0.000,1,0,0,0


In [ ]:
dt.shape

(35887, 8)

In [ ]:
dt.describe()


,travisJobId,index,duration,count,failures,errors,skipped
count,3.588700e+04,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000,35887.000000
mean,8.584269e+07,19.230251,0.114179,6.722741,0.086271,0.109678,0.178143
std,3.224883e+07,11.780563,4.671694,17.726826,1.183693,1.994337,0.430674
min,1.111925e+06,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,8.960691e+07,9.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,9.071805e+07,19.000000,0.001000,1.000000,0.000000,0.000000,0.000000
75%,9.436153e+07,28.000000,0.010000,3.000000,0.000000,0.000000,0.000000
max,1.422286e+08,43.000000,572.163000,103.000000,66.000000,93.000000,2.000000


In [ ]:
dt.columns

Index(['travisJobId', 'testName', 'index', 'duration', 'count', 'failures',
       'errors', 'skipped'],
      dtype='object')

In [ ]:
dt.nunique()

travisJobId     932
testName         46
index            44
duration       1324
count            73
failures         29
errors           30
skipped           3
dtype: int64

In [ ]:
dt['index'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43])

In [ ]:
dt['duration'].unique()

array([0.17 , 0.05 , 0.013, ..., 1.391, 1.159, 0.596])

In [ ]:
dt['failures'].unique()

array([ 1,  0,  6, 11, 18,  4,  2, 17,  7, 34, 12, 19, 31, 22, 24, 66, 16,
        5,  9, 43, 14, 25, 36, 27, 15, 20, 10, 28, 21])

In [ ]:
binarized = preprocessing.Binarizer(threshold=1).transform(np.array(dt['failures']).reshape(-1,1))

dt['failures'].unique()



array([ 1,  0,  6, 11, 18,  4,  2, 17,  7, 34, 12, 19, 31, 22, 24, 66, 16,
        5,  9, 43, 14, 25, 36, 27, 15, 20, 10, 28, 21])

In [ ]:
dt['failures'] = binarized

dt['failures'].unique()

array([0, 1])

In [ ]:
#corelation = dt.corr()
#sns.heatmap(corelation, xticklabels = corelation.columns, yticklabels = corelation.columns, annot = True)
#sns.pairplot(dt)
#sns.distplot(data['failures'])
#sns.distplot(data['count'])
#sns.distplot(data['duration'])
#sns.catplot(x='failures', kind='box', data=data)

In [ ]:
print(len(dt['testName'].unique()))

46


In [ ]:
df=pd.DataFrame(dt)

ord_enc = OrdinalEncoder()
df["test_code"] = ord_enc.fit_transform(df[["testName"]])

df.head()

,travisJobId,testName,index,duration,count,failures,errors,skipped,test_code
0,87853794,de.neuland.jade4j.exceptions.JadeExceptionTest,0,0.170,2,0,0,0,8.0
1,87853794,de.neuland.jade4j.template.JadeRunFullTemplate...,1,0.050,2,0,0,0,42.0
2,87853794,de.neuland.jade4j.template.JadeConfigurationTest,2,0.013,5,0,0,0,41.0
3,87853794,de.neuland.jade4j.model.JadeModelTest,3,0.000,2,0,0,0,25.0
4,87853794,de.neuland.jade4j.parser.ComplexIndentOutdentP...,4,0.015,1,0,0,0,28.0


In [ ]:
input_x = df.drop(['travisJobId','failures', 'testName'], axis = 1)
input_x.head()


,index,duration,count,errors,skipped,test_code
0,0,0.170,2,0,0,8.0
1,1,0.050,2,0,0,42.0
2,2,0.013,5,0,0,41.0
3,3,0.000,2,0,0,25.0
4,4,0.015,1,0,0,28.0


In [ ]:
input_x.isnull().any()

index        False
duration     False
count        False
errors       False
skipped      False
test_code    False
dtype: bool

In [ ]:
output_y = df['failures']
output_y.head()

0    0
1    0
2    0
3    0
4    0
Name: failures, dtype: int64

In [ ]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(input_x, output_y, test_size=0.7)

#print(train_x, test_x, train_y, test_y)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)



(10766, 6) (25121, 6) (10766,) (25121,)


In [ ]:
#Classifier1 
clf1 = GaussianNB()
clf1.fit(train_x, train_y)
predictions1 = clf1.predict(test_x)
print("\nAccuracy score ",accuracy_score(test_y, predictions1))
print("\nConfusion_matrix",confusion_matrix(test_y, predictions1))
print("\nClassification_report",classification_report(test_y, predictions1))
prob1 = clf1.predict_proba(test_x)
# Keeping only the positive class
prob1 = [p[1] for p in prob1]
print("\nROC score ",roc_auc_score(test_y, prob1))

df_proba1 = pd.DataFrame(clf1.predict_proba(test_x), columns=clf1.classes_)
df_out1 = pd.merge(df,df_proba1,how = 'left',left_index = True, right_index = True)
df_out1.head()

dfsort1 = df_out1.sort_values(by=[0,1])
print(dfsort1)

from google.colab import drive
drive.mount('/content/gdrive')

dfsort1.to_csv(r'/content/gdrive/MyDrive/Colab Notebooks/SQ/jade4j@nb.csv')


Accuracy score  0.9640937860753951

Confusion_matrix [[24119   830]
 [   72   100]]

Classification_report               precision    recall  f1-score   support

           0       1.00      0.97      0.98     24949
           1       0.11      0.58      0.18       172

    accuracy                           0.96     25121
   macro avg       0.55      0.77      0.58     25121
weighted avg       0.99      0.96      0.98     25121


ROC score  0.9056314416292959
       travisJobId                                           testName  ...    0    1
280       42504092    de.neuland.jade4j.compiler.CompilerTagErrorTest  ...  0.0  1.0
489       11749504  de.neuland.jade4j.compiler.CompilerIndentation...  ...  0.0  1.0
1781      90260783     de.neuland.jade4j.exceptions.JadeExceptionTest  ...  0.0  1.0
2144      90338663              de.neuland.jade4j.lexer.token.TagTest  ...  0.0  1.0
2221      42506595   de.neuland.jade4j.parser.LargeBodyTextParserTest  ...  0.0  1.0
...            ...      

In [ ]:
#Classifier2
clf2 = LogisticRegression()
clf2.fit(train_x, train_y)
predictions2 = clf2.predict(test_x)
print("\nAccuracy score ",accuracy_score(test_y, predictions2))
print("\nConfusion_matrix",confusion_matrix(test_y, predictions2))
print("\nClassification_report",classification_report(test_y, predictions2))
prob2 = clf2.predict_proba(test_x)
# Keeping only the positive class
prob2 = [p[1] for p in prob2]
print("\nROC score ",roc_auc_score(test_y, prob2))

df_proba2 = pd.DataFrame(clf2.predict_proba(test_x), columns=clf2.classes_)
df_out2 = pd.merge(df,df_proba2,how = 'left',left_index = True, right_index = True)
df_out2.head()

dfsort2 = df_out2.sort_values(by=[0,1])
print(dfsort2)

#from google.colab import drive
#drive.mount('/content/gdrive')

dfsort2.to_csv(r'/content/gdrive/MyDrive/Colab Notebooks/SQ/jade4j@lr.csv')



Accuracy score  0.9963377254090203

Confusion_matrix [[24948     1]
 [   91    81]]

Classification_report               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24949
           1       0.99      0.47      0.64       172

    accuracy                           1.00     25121
   macro avg       0.99      0.74      0.82     25121
weighted avg       1.00      1.00      1.00     25121


ROC score  0.90015876574258
       travisJobId                                           testName  ...    0    1
280       42504092    de.neuland.jade4j.compiler.CompilerTagErrorTest  ...  0.0  1.0
489       11749504  de.neuland.jade4j.compiler.CompilerIndentation...  ...  0.0  1.0
1781      90260783     de.neuland.jade4j.exceptions.JadeExceptionTest  ...  0.0  1.0
2144      90338663              de.neuland.jade4j.lexer.token.TagTest  ...  0.0  1.0
2221      42506595   de.neuland.jade4j.parser.LargeBodyTextParserTest  ...  0.0  1.0
...            ...        

In [ ]:
#Classifier3
clf3 = RandomForestClassifier()
clf3.fit(train_x, train_y)
predictions3 = clf3.predict(test_x)
print("\nAccuracy score ",accuracy_score(test_y, predictions3))
print("\nConfusion_matrix",confusion_matrix(test_y, predictions3))
print("\nClassification_report",classification_report(test_y, predictions3))
prob3 = clf3.predict_proba(test_x)
# Keeping only the positive class
prob3 = [p[1] for p in prob3]
print("\nROC score ",roc_auc_score(test_y, prob3))

df_proba3 = pd.DataFrame(clf3.predict_proba(test_x), columns=clf3.classes_)
df_out3 = pd.merge(df,df_proba3,how = 'left',left_index = True, right_index = True)
df_out3.head()

dfsort3 = df_out3.sort_values(by=[0,1])
print(dfsort3)

dfsort3.to_csv(r'/content/gdrive/MyDrive/Colab Notebooks/SQ/jade4j@rf.csv')


Accuracy score  0.9979300187094463

Confusion_matrix [[24942     7]
 [   45   127]]

Classification_report               precision    recall  f1-score   support

           0       1.00      1.00      1.00     24949
           1       0.95      0.74      0.83       172

    accuracy                           1.00     25121
   macro avg       0.97      0.87      0.91     25121
weighted avg       1.00      1.00      1.00     25121


ROC score  0.954392076114343
       travisJobId                                           testName  ...    0    1
280       42504092    de.neuland.jade4j.compiler.CompilerTagErrorTest  ...  0.0  1.0
489       11749504  de.neuland.jade4j.compiler.CompilerIndentation...  ...  0.0  1.0
1781      90260783     de.neuland.jade4j.exceptions.JadeExceptionTest  ...  0.0  1.0
2144      90338663              de.neuland.jade4j.lexer.token.TagTest  ...  0.0  1.0
2982      93791899           de.neuland.jade4j.lexer.token.ForTagTest  ...  0.0  1.0
...            ...       